This notebook demonstrates the Finite-Interval Forecasting Engine (FIFE).

FIFE is a Python package developed by the Institute for Defense Analyses in a research partnership with the U.S. Office of the Undersecretary of Defense for Personnel and Readiness.

FIFE applies state-of-the-art forecasting methods to [panel data](https://en.wikipedia.org/wiki/Panel_data). A panel dataset contains periodic observations of subjects. In a given period, subjects may enter or exit the panel; subjects that do not exit survive to be observed in later periods. FIFE can forecast subject-specific probabilities of survival, future states of survival, and circumstances of exit for each future period up to a given or inferred maximum time horizon.

In this example we forecast when each leader of the world's countries will lose power. To skip past how FIFE works to what FIFE can produce, see the [Forecasts section](#Forecasts).

In [1]:
from fife.utils import make_results_reproducible
from fife.processors import PanelDataProcessor
from fife.lgb_modelers import LGBSurvivalModeler
import pandas as pd

Let's make sure we get the same results each time we execute this code from top to bottom.

In [2]:
SEED = 9999
make_results_reproducible(SEED)

We use the July 2020 edition of the [Rulers, Elections, and Irregular Governance dataset (REIGN)](https://oefdatascience.github.io/REIGN.github.io/), a monthly panel of national leaders and political conditions since January 1950. We load the REIGN data directly from its online archive. If you plan to execute this notebook repeatedly, please avoid spamming the REIGN archive with requests by saving the data locally, then changing the file path in the cell below to your local path.

In [3]:
data = pd.read_csv("https://www.dl.dropboxusercontent.com/s/3tdswu2jfgwp4xw/REIGN_2020_7.csv?dl=0")
data.head()

,ccode,country,leader,year,month,elected,age,male,militarycareer,tenure_months,...,delayed,lastelection,loss,irregular,prev_conflict,pt_suc,pt_attempt,precip,couprisk,pctile_risk
0,2.0,USA,Truman,1950.0,1.0,1.0,66.0,1,0.0,58.0,...,0.0,2.639057,5.327876,7.565793,0.0,0.0,0.0,-0.069058,NaN,NaN
1,2.0,USA,Truman,1950.0,2.0,1.0,66.0,1,0.0,59.0,...,0.0,2.708050,5.332719,7.566311,0.0,0.0,0.0,-0.113721,NaN,NaN
2,2.0,USA,Truman,1950.0,3.0,1.0,66.0,1,0.0,60.0,...,0.0,2.772589,5.337538,7.566829,0.0,0.0,0.0,-0.108042,NaN,NaN
3,2.0,USA,Truman,1950.0,4.0,1.0,66.0,1,0.0,61.0,...,0.0,2.833213,5.342334,7.567346,0.0,0.0,0.0,-0.041600,NaN,NaN
4,2.0,USA,Truman,1950.0,5.0,1.0,66.0,1,0.0,62.0,...,0.0,2.890372,5.347107,7.567863,0.0,0.0,0.0,-0.129703,NaN,NaN


The data need only minimal pre-processing to be ready for FIFE. Panel data are defined by two identifiers: individual and time. In the REIGN data, we can identify an individual by their name and the country they lead. We can identify time by year and month. FIFE will automatically infer the individual and time identifiers as the two left-most columns in the data, so we rearrange the columns accordingly. Along the way we drop the redundant columns `ccode` (a numeric code for country) and `leader`.

In [4]:
data["country-leader"] = data["country"] + ": " + data["leader"]
data["year-month"] = data["year"].astype(int).astype(str) + data["month"].astype(int).astype(str).str.zfill(2)
data["year-month"] = pd.to_datetime(data["year-month"], format="%Y%m")
data = pd.concat([data[["country-leader", "year-month"]],
                  data.drop(["ccode", "country-leader", "leader", "year-month"],
                            axis=1)],
                 axis=1)
data.head()

,country-leader,year-month,country,year,month,elected,age,male,militarycareer,tenure_months,...,delayed,lastelection,loss,irregular,prev_conflict,pt_suc,pt_attempt,precip,couprisk,pctile_risk
0,USA: Truman,1950-01-01,USA,1950.0,1.0,1.0,66.0,1,0.0,58.0,...,0.0,2.639057,5.327876,7.565793,0.0,0.0,0.0,-0.069058,NaN,NaN
1,USA: Truman,1950-02-01,USA,1950.0,2.0,1.0,66.0,1,0.0,59.0,...,0.0,2.708050,5.332719,7.566311,0.0,0.0,0.0,-0.113721,NaN,NaN
2,USA: Truman,1950-03-01,USA,1950.0,3.0,1.0,66.0,1,0.0,60.0,...,0.0,2.772589,5.337538,7.566829,0.0,0.0,0.0,-0.108042,NaN,NaN
3,USA: Truman,1950-04-01,USA,1950.0,4.0,1.0,66.0,1,0.0,61.0,...,0.0,2.833213,5.342334,7.567346,0.0,0.0,0.0,-0.041600,NaN,NaN
4,USA: Truman,1950-05-01,USA,1950.0,5.0,1.0,66.0,1,0.0,62.0,...,0.0,2.890372,5.347107,7.567863,0.0,0.0,0.0,-0.129703,NaN,NaN


By definition, each unique pair of identifier values identifies a single observation. REIGN contains a few violations of this definition, due to leaders losing and regaining power in a given month. We keep only the last observation of a given leader in a given month, thereby ignoring intra-month lapses in power.

In [5]:
total_obs = len(data)
data = data.drop_duplicates(["country-leader", "year-month"], keep="last")
n_duplicates = total_obs - len(data)
print(f"{n_duplicates} observations with a duplicated identifier pair deleted.")

7 observations with a duplicated identifier pair deleted.


FIFE's PanelDataProcessor will do the rest of our data processing for us, including dropping any degenerate features, identifying which features are categorical, and computing which leadership spells ended and when.

In [6]:
data_processor = PanelDataProcessor(data=data)
data_processor.build_processed_data()

Time identifier column name not given; assumed to be second-leftmost column (year-month)
Individual identifier column name not given; assumed to be leftmost column (country-leader)


We pass the processed data to FIFE's LGBSurvivalModeler, which trains gradient-boosted tree models using [LightGBM](https://lightgbm.readthedocs.io/en/latest/). FIFE trains one model for each time horizon up to some maximum. In the absence of a specified maximum, FIFE will train up to the time horizon for which there are no more than 64 training observations that survived that long.

In [7]:
modeler = LGBSurvivalModeler(data=data_processor.data)
modeler.build_model()

Time identifier column name not given; assumed to be second-leftmost column (year-month)
Individual identifier column name not given; assumed to be leftmost column (country-leader)


Found `num_iterations` in params. Will use it instead of argument
categorical_feature in Dataset is overridden.
New categorical_feature is ['age', 'anticipation', 'change_recent', 'country', 'defeat_recent', 'delayed', 'direct_recent', 'elected', 'election_now', 'election_recent', 'exec_ant', 'exec_recent', 'government', 'indirect_recent', 'irreg_lead_ant', 'lead_recent', 'leg_ant', 'leg_recent', 'male', 'militarycareer', 'month', 'nochange_recent', 'prev_conflict', 'pt_attempt', 'pt_suc', 'ref_ant', 'ref_recent', 'tenure_months', 'victory_recent', 'year']


[1]	validation_set's binary_logloss: 0.0450614
Training until validation scores don't improve for 4 rounds
[1]	validation_set's binary_logloss: 0.0446671[2]	validation_set's binary_logloss: 0.0454517
Training until validation scores don't improve for 4 rounds

[3]	validation_set's binary_logloss: 0.045389
[2]	validation_set's binary_logloss: 0.0451477
[4]	validation_set's binary_logloss: 0.0463014[3]	validation_set's binary_logloss: 0.0455293

[4]	validation_set's binary_logloss: 0.0457212
[5]	validation_set's binary_logloss: 0.0464527
[5]	validation_set's binary_logloss: 0.0461752Early stopping, best iteration is:
[1]	validation_set's binary_logloss: 0.0450614

Early stopping, best iteration is:
[1]	validation_set's binary_logloss: 0.0446671
[1]	validation_set's binary_logloss: 0.0494178
Training until validation scores don't improve for 4 rounds
[1]	validation_set's binary_logloss: 0.0516978[2]	validation_set's binary_logloss: 0.0516761

Training until validation scores don't improve

Found `num_iterations` in params. Will use it instead of argument
categorical_feature in Dataset is overridden.
New categorical_feature is ['age', 'anticipation', 'change_recent', 'country', 'defeat_recent', 'delayed', 'direct_recent', 'elected', 'election_now', 'election_recent', 'exec_ant', 'exec_recent', 'government', 'indirect_recent', 'irreg_lead_ant', 'lead_recent', 'leg_ant', 'leg_recent', 'male', 'militarycareer', 'month', 'nochange_recent', 'prev_conflict', 'pt_attempt', 'pt_suc', 'ref_ant', 'ref_recent', 'tenure_months', 'victory_recent', 'year']


[1]	validation_set's binary_logloss: 0.0666623
Training until validation scores don't improve for 4 rounds
[2]	validation_set's binary_logloss: 0.0660299

...

[8]	validation_set's binary_logloss: 0.0613905

[3]	validation_set's binary_logloss: 0.049618[9]	validation_set's binary_logloss: 0.0612873

Early stopping, best iteration is:
[5]	validation_set's binary_logloss: 0.0611482
[4]	validation_set's binary_logloss: 0.0497081
[5]	validation_set's binary_logloss: 0.0502232
Early stopping, best iteration is:
[1]	validation_set's binary_logloss: 0.0456362
[1]	validation_set's binary_logloss: 0.0471149
Training until validation scores don't improve for 4 rounds
[2]	validation_set's binary_logloss: 0.0476308
[3]	validation_set's binary_logloss: 0.0482606
[4]	validation_set's binary_logloss: 0.0485808
[5]	validation_set's binary_logloss: 0.0489201
Early stopping, best iteration is:
[1]	validation_set's binary_logloss: 0.0471149


Found `num_iterations` in params. Will use it instead of argument
categorical_feature in Dataset is overridden.
New categorical_feature is ['age', 'anticipation', 'change_recent', 'country', 'defeat_recent', 'delayed', 'direct_recent', 'elected', 'election_now', 'election_recent', 'exec_ant', 'exec_recent', 'government', 'indirect_recent', 'irreg_lead_ant', 'lead_recent', 'leg_ant', 'leg_recent', 'male', 'militarycareer', 'month', 'nochange_recent', 'prev_conflict', 'pt_attempt', 'pt_suc', 'ref_ant', 'ref_recent', 'tenure_months', 'victory_recent', 'year']


[1]	validation_set's binary_logloss: 0.0492351
Training until validation scores don't improve for 4 rounds
[2]	validation_set's binary_logloss: 0.0495282
[3]	validation_set's binary_logloss: 0.0499765
[1]	validation_set's binary_logloss: 0.0449602
[4]	validation_set's binary_logloss: 0.0497722Training until validation scores don't improve for 4 rounds
[5]	validation_set's binary_logloss: 0.0502406

Early stopping, best iteration is:
[1]	validation_set's binary_logloss: 0.0492351[2]	validation_set's binary_logloss: 0.0461643

[3]	validation_set's binary_logloss: 0.0461854
[4]	validation_set's binary_logloss: 0.0464649
[5]	validation_set's binary_logloss: 0.0462984
Early stopping, best iteration is:
[1]	validation_set's binary_logloss: 0.0449602


Found `num_iterations` in params. Will use it instead of argument
categorical_feature in Dataset is overridden.
New categorical_feature is ['age', 'anticipation', 'change_recent', 'country', 'defeat_recent', 'delayed', 'direct_recent', 'elected', 'election_now', 'election_recent', 'exec_ant', 'exec_recent', 'government', 'indirect_recent', 'irreg_lead_ant', 'lead_recent', 'leg_ant', 'leg_recent', 'male', 'militarycareer', 'month', 'nochange_recent', 'prev_conflict', 'pt_attempt', 'pt_suc', 'ref_ant', 'ref_recent', 'tenure_months', 'victory_recent', 'year']


[1]	validation_set's binary_logloss: 0.0484633
Training until validation scores don't improve for 4 rounds
[2]	validation_set's binary_logloss: 0.0483523
[3]	validation_set's binary_logloss: 0.0486443
[4]	validation_set's binary_logloss: 0.049105
[5]	validation_set's binary_logloss: 0.049426
[6]	validation_set's binary_logloss: 0.0498055
Early stopping, best iteration is:
[2]	validation_set's binary_logloss: 0.0483523
[1]	validation_set's binary_logloss: 0.0579718
Training until validation scores don't improve for 4 rounds
[2]	validation_set's binary_logloss: 0.0574876
[3]	validation_set's binary_logloss: 0.0576595
[1]	validation_set's binary_logloss: 0.0414249[4]	validation_set's binary_logloss: 0.057774

Training until validation scores don't improve for 4 rounds
[2]	validation_set's binary_logloss: 0.0419145[5]	validation_set's binary_logloss: 0.0578908

[3]	validation_set's binary_logloss: 0.0421709[6]	validation_set's binary_logloss: 0.0579768

Early stopping, best iteration is:
[

#### Forecasts
Obtaining a forecast for every current leader is as simple as `modeler.forecast()`. We also customize the column headers.

In [8]:
forecasts = modeler.forecast()
forecasts.columns = pd.date_range(data["year-month"].max(),
                                  periods=len(forecasts.columns) + 1,
                                  freq="M",
                                  ).strftime("%b %Y")[1:]
forecasts

,Aug 2020,Sep 2020,Oct 2020,Nov 2020,Dec 2020,Jan 2021,Feb 2021,Mar 2021,Apr 2021,May 2021,...,Dec 2039,Jan 2040,Feb 2040,Mar 2040,Apr 2040,May 2040,Jun 2040,Jul 2040,Aug 2040,Sep 2040
country-leader,,,,,,,,,,,,,,,,,,,,,
Afghanistan: Ashraf Ghani,0.993510,0.972192,0.947423,0.928553,0.910774,0.903915,0.898858,0.881401,0.875547,0.865219,...,0.021501,0.020679,0.020542,0.020289,0.018357,0.018235,0.018112,0.018009,0.017885,0.017775
Albania: Rama,0.994266,0.987576,0.979449,0.975458,0.970171,0.965339,0.959064,0.952718,0.943017,0.929097,...,0.004326,0.004298,0.004252,0.004223,0.004127,0.004099,0.004071,0.004048,0.004020,0.003990
Algeria: Tebboune,0.981568,0.954030,0.943877,0.930683,0.922343,0.913518,0.906216,0.900997,0.892311,0.880333,...,0.104825,0.104140,0.103450,0.102745,0.102051,0.101369,0.100686,0.100117,0.099428,0.098814
Andorra: Espot Zamora,0.988504,0.980634,0.973549,0.964426,0.955670,0.947639,0.934887,0.928751,0.920484,0.911905,...,0.020801,0.020665,0.020528,0.020388,0.020251,0.020115,0.019980,0.019867,0.019730,0.019575
Angola: Lourenco,0.986212,0.971566,0.963054,0.958269,0.951608,0.939726,0.935119,0.924575,0.901211,0.890643,...,0.062187,0.061780,0.061373,0.060954,0.058901,0.058508,0.058114,0.057786,0.057388,0.057027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Venezuela: Nicolas Maduro,0.996550,0.992178,0.988821,0.986147,0.982919,0.979942,0.976554,0.971374,0.942708,0.936070,...,0.081647,0.081111,0.080574,0.078506,0.076717,0.076216,0.075702,0.075275,0.074757,0.074295
Vietnam: Phu Trong,0.992936,0.987022,0.981365,0.910920,0.903252,0.893244,0.887394,0.875696,0.821941,0.816099,...,0.053048,0.052687,0.052328,0.051971,0.051620,0.051276,0.048573,0.048219,0.047887,0.047586
Yemen: Houthi,0.997050,0.982350,0.971474,0.959179,0.952695,0.942669,0.935131,0.928477,0.915261,0.903341,...,0.117514,0.116743,0.115970,0.115179,0.114401,0.113637,0.110038,0.109418,0.108681,0.108026


We won't know how accurate these forecasts are until the future reveals itself to us. However, we can *pretend* to not have the most recent, say, 36 months of data, train a model, then evaluate that model's forecasts over those 36 months. Doing so is as easy as specifying the TEST_INTERVALS configuration parameter.

In [9]:
TEST_INTERVALS = 36
data_processor = PanelDataProcessor(config={"TEST_INTERVALS": TEST_INTERVALS}, data=data)
data_processor.build_processed_data()

modeler = LGBSurvivalModeler(data=data_processor.data)
modeler.build_model(TEST_INTERVALS)

Time identifier column name not given; assumed to be second-leftmost column (year-month)
Individual identifier column name not given; assumed to be leftmost column (country-leader)


Found `num_iterations` in params. Will use it instead of argument
categorical_feature in Dataset is overridden.
New categorical_feature is ['age', 'anticipation', 'change_recent', 'country', 'defeat_recent', 'delayed', 'direct_recent', 'elected', 'election_now', 'election_recent', 'exec_ant', 'exec_recent', 'government', 'indirect_recent', 'irreg_lead_ant', 'lead_recent', 'leg_ant', 'leg_recent', 'male', 'militarycareer', 'month', 'nochange_recent', 'prev_conflict', 'pt_attempt', 'pt_suc', 'ref_ant', 'ref_recent', 'tenure_months', 'victory_recent', 'year']


[1]	validation_set's binary_logloss: 0.0644855
Training until validation scores don't improve for 4 rounds
[2]	validation_set's binary_logloss: 0.0639783
[3]	validation_set's binary_logloss: 0.0637411
[4]	validation_set's binary_logloss: 0.0634981[1]	validation_set's binary_logloss: 0.0638733[1]	validation_set's binary_logloss: 0.0636556[1]	validation_set's binary_logloss: 0.0654157[1]	validation_set's binary_logloss: 0.064328
[1]	validation_set's binary_logloss: 0.0647894




Training until validation scores don't improve for 4 roundsTraining until validation scores don't improve for 4 roundsTraining until validation scores don't improve for 4 roundsTraining until validation scores don't improve for 4 rounds
Training until validation scores don't improve for 4 rounds



[1]	validation_set's binary_logloss: 0.0647049[1]	validation_set's binary_logloss: 0.0772667

Training until validation scores don't improve for 4 roundsTraining until validation scores don't improve for 4 rounds[2]	va

Found `num_iterations` in params. Will use it instead of argument


[19]	validation_set's binary_logloss: 0.0530511

...

[26]	validation_set's binary_logloss: 0.0626955[8]	validation_set's binary_logloss: 0.0637235

[27]	validation_set's binary_logloss: 0.0625931[9]	validation_set's binary_logloss: 0.0638662

[28]	validation_set's binary_logloss: 0.0624695
[10]	validation_set's binary_logloss: 0.0639151
[29]	validation_set's binary_logloss: 0.0624466[11]	validation_set's binary_logloss: 0.0640306

Early stopping, best iteration is:
[7]	validation_set's binary_logloss: 0.0636318
[30]	validation_set's binary_logloss: 0.0624387
[31]	validation_set's binary_logloss: 0.0624377
[32]	validation_set's binary_logloss: 0.062459
[33]	validation_set's binary_logloss: 0.0624414
[34]	validation_set's binary_logloss: 0.0625505


Found `num_iterations` in params. Will use it instead of argument
categorical_feature in Dataset is overridden.
New categorical_feature is ['age', 'anticipation', 'change_recent', 'country', 'defeat_recent', 'delayed', 'direct_recent', 'elected', 'election_now', 'election_recent', 'exec_ant', 'exec_recent', 'government', 'indirect_recent', 'irreg_lead_ant', 'lead_recent', 'leg_ant', 'leg_recent', 'male', 'militarycareer', 'month', 'nochange_recent', 'prev_conflict', 'pt_attempt', 'pt_suc', 'ref_ant', 'ref_recent', 'tenure_months', 'victory_recent', 'year']


[35]	validation_set's binary_logloss: 0.0625522
Early stopping, best iteration is:
[31]	validation_set's binary_logloss: 0.0624377
[1]	validation_set's binary_logloss: 0.0650748
Training until validation scores don't improve for 4 rounds
[2]	validation_set's binary_logloss: 0.0645201
[3]	validation_set's binary_logloss: 0.0644441
[4]	validation_set's binary_logloss: 0.0642014
[5]	validation_set's binary_logloss: 0.0641898
[6]	validation_set's binary_logloss: 0.0640868


Found `num_iterations` in params. Will use it instead of argument
categorical_feature in Dataset is overridden.
New categorical_feature is ['age', 'anticipation', 'change_recent', 'country', 'defeat_recent', 'delayed', 'direct_recent', 'elected', 'election_now', 'election_recent', 'exec_ant', 'exec_recent', 'government', 'indirect_recent', 'irreg_lead_ant', 'lead_recent', 'leg_ant', 'leg_recent', 'male', 'militarycareer', 'month', 'nochange_recent', 'prev_conflict', 'pt_attempt', 'pt_suc', 'ref_ant', 'ref_recent', 'tenure_months', 'victory_recent', 'year']


[7]	validation_set's binary_logloss: 0.0639292
[8]	validation_set's binary_logloss: 0.0640105
[9]	validation_set's binary_logloss: 0.0640963
[10]	validation_set's binary_logloss: 0.0639629
[11]	validation_set's binary_logloss: 0.0639413
Early stopping, best iteration is:
[7]	validation_set's binary_logloss: 0.0639292
[1]	validation_set's binary_logloss: 0.0726028[1]	validation_set's binary_logloss: 0.0679752

Training until validation scores don't improve for 4 roundsTraining until validation scores don't improve for 4 rounds

[2]	validation_set's binary_logloss: 0.0707853[2]	validation_set's binary_logloss: 0.0676806

[3]	validation_set's binary_logloss: 0.0673609
[3]	validation_set's binary_logloss: 0.0699883
[4]	validation_set's binary_logloss: 0.0669555
[4]	validation_set's binary_logloss: 0.0692417[5]	validation_set's binary_logloss: 0.0666383

[6]	validation_set's binary_logloss: 0.0664538[5]	validation_set's binary_logloss: 0.0686643

[7]	validation_set's binary_logloss: 0.06606

We want to evaluate our model on the set of leaders in July 2017, 36 months prior to the most recent month in our dataset.

In [10]:
evaluation_subset = data_processor.data["_period"] == (data_processor.data["_period"].max() - TEST_INTERVALS)
metrics = modeler.evaluate(evaluation_subset)
metrics

,AUROC,Predicted Share,Actual Share,True Positives,False Negatives,False Positives,True Negatives,Other Metrics:,C-Index
Lead Length,,,,,,,,,
1,0.848014,0.991015,0.984694,193,0,3,0,,0.7039644012944983
2,NaN,0.982337,1.000000,193,0,0,0,,
3,0.536649,0.971161,0.989637,191,0,2,0,,
4,0.978836,0.952692,0.989529,188,1,1,1,,
5,0.716304,0.945227,0.973545,183,1,5,0,,
6,0.950549,0.931157,0.989130,181,1,1,1,,
7,0.988950,0.920197,0.994505,179,2,1,0,,
8,0.871508,0.905779,0.988950,176,3,1,1,,
9,0.694286,0.892268,0.977654,172,3,3,1,,


The model we trained used default values for hyperparameters such as the maximum number of leaves per tree and the minimum number of observations in each leaf. There are likely other hyperparameter values that would yield better performance for each future period. Searching for those values and returning the best candidate set for each future period (according to performance on a validation set) is as simple as `modeler.hyperoptimize()`. The number of sets to consider, `n_trials`, defaults to 64, but we'll specify a value of 4 to accelerate this example.

In [11]:
N_TRIALS = 4
params = modeler.hyperoptimize(n_trials=N_TRIALS)

[I 2020-08-27 17:19:37,117] Trial 0 finished with value: 0.06042667685953438 and parameters: {'num_iterations': 94, 'learning_rate': 0.4032261012568846, 'num_leaves': 121, 'max_depth': 19, 'min_data_in_leaf': 369, 'min_sum_hessian_in_leaf': 0.1364906231563803, 'bagging_freq': 0, 'bagging_fraction': 0.7271848812213663, 'feature_fraction': 0.6779042758880096, 'lambda_l1': 55.32233457830071, 'lambda_l2': 42.84656691186624, 'min_gain_to_split': 0.07127895884983065, 'min_data_per_group': 431, 'max_cat_threshold': 70, 'cat_l2': 41.37300385050969, 'cat_smooth': 1229.1342054498475, 'max_cat_to_onehot': 16, 'max_bin': 326, 'min_data_in_bin': 37}. Best is trial 0 with value: 0.06042667685953438.
[I 2020-08-27 17:19:37,763] Trial 1 finished with value: 0.060577645587346615 and parameters: {'num_iterations': 67, 'learning_rate': 0.03746294609752311, 'num_leaves': 158, 'max_depth': 12, 'min_data_in_leaf': 32, 'min_sum_hessian_in_leaf': 0.14511985173887948, 'bagging_freq': 0, 'bagging_fraction': 0.6

Using our new hyperparameter sets is as simple as passing them to build_model.

In [13]:
modeler.build_model(TEST_INTERVALS, params)

Found `num_iterations` in params. Will use it instead of argument
categorical_feature in Dataset is overridden.
New categorical_feature is ['age', 'anticipation', 'change_recent', 'country', 'defeat_recent', 'delayed', 'direct_recent', 'elected', 'election_now', 'election_recent', 'exec_ant', 'exec_recent', 'government', 'indirect_recent', 'irreg_lead_ant', 'lead_recent', 'leg_ant', 'leg_recent', 'male', 'militarycareer', 'month', 'nochange_recent', 'prev_conflict', 'pt_attempt', 'pt_suc', 'ref_ant', 'ref_recent', 'tenure_months', 'victory_recent', 'year']
Found `num_iterations` in params. Will use it instead of argument
categorical_feature in Dataset is overridden.
New categorical_feature is ['age', 'anticipation', 'change_recent', 'country', 'defeat_recent', 'delayed', 'direct_recent', 'elected', 'election_now', 'election_recent', 'exec_ant', 'exec_recent', 'government', 'indirect_recent', 'irreg_lead_ant', 'lead_recent', 'leg_ant', 'leg_recent', 'male', 'militarycareer', 'month', 'n

We can now evaluate our new model and directly compare it to our default model. The best hyperparameters on a validation set don't always outperform the default parameters on the test set, but in this case they do.

In [14]:
hyperoptimized_metrics = modeler.evaluate(evaluation_subset)
hyperoptimized_metrics

,AUROC,Predicted Share,Actual Share,True Positives,False Negatives,False Positives,True Negatives,Other Metrics:,C-Index
Lead Length,,,,,,,,,
1,0.911917,0.984838,0.984694,192,1,3,0,,0.7291262135922331
2,NaN,0.974906,1.000000,192,1,0,0,,
3,0.554974,0.964083,0.989637,191,0,1,1,,
4,0.994709,0.945375,0.989529,189,0,1,1,,
5,0.721739,0.932101,0.973545,183,1,5,0,,
6,0.975275,0.914116,0.989130,181,1,1,1,,
7,0.977901,0.905794,0.994505,178,3,1,0,,
8,0.824022,0.894197,0.988950,174,5,1,1,,
9,0.812857,0.881855,0.977654,171,4,2,2,,
